In [1]:
import pandas as pd
import csv
import re
import string
import nltk

from pythainlp.tokenize import Tokenizer
from pythainlp import word_tokenize
from pythainlp.corpus import thai_stopwords
from pythainlp.tokenize import dict_trie
from pythainlp.tag import pos_tag

from nltk.corpus import words
from stop_words import get_stop_words

In [2]:
# word correction for EN
#parser = GingerIt()

# download stopwords of TH and EN
nltk.download('words')
th_stop = tuple(thai_stopwords())
en_stop = tuple(get_stop_words('en'))

# clean the text
def clean_text(msg): 

    msg = str(msg).lower()
    msg = re.sub(r"\,", "", str(msg))
    msg = re.sub(r"[^a-z0-9ก-๙\.\%\:]", " ", str(msg))
    
    # Delete punctuation
   # str_punc = string.punctuation.replace(".","")
   # for c in str_punc:
    #    msg = re.sub(r'\{}'.format(c),'',str(msg))
    
    # ลบ separator เช่น \n \t
    msg = ' '.join(msg.split())
    
    return msg

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\super\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [8]:
# input
# import data from csv
file_name = 'data_jib_1'

FILE = './Data/{}.csv'.format(file_name)
df = pd.read_csv(FILE,index_col=0)

In [10]:
df.head()

,P_0Q,P_0A,P_1Q,P_1A,P_2Q,P_2A,P_3Q,P_3A,P_4Q,P_4A,...,P_3882Q,P_3882A,P_3883Q,P_3883A,P_3884Q,P_3884A,P_3885Q,P_3885A,P_3886Q,P_3886A
0,CPU Brand,AMD,CPU Brand,Intel,CPU Brand,AMD,CPU Brand,Intel,CPU Brand,Intel,...,Model,INTEL 1151 CORE I5-9400F 2.90 GHz,Model,INTEL 1151 CORE I5-9600KF 3.70 GHz,Model,AMD AM4 RYZEN 5 3600 3.6GHz,Model,CPU INTEL 1151 CORE I7-9700KF,Model,AMD AM4 RYZEN 7 3700X 3.6GHz
1,CPU Model,AMD Ryzen 5 3500U,CPU Model,Intel Core i5-10210U,CPU Model,AMD Ryzen 5 3500U,CPU Model,Intel Core i5-1035G1,CPU Model,Intel Core i5-9300H,...,CPU Core / Thread,6/6,CPU Core / Thread,6/6,CPU Core / Thread,6/12,CPU Core / Thread,8/8,CPU Core / Thread,8/16
2,Base Clock,2.00 GHz,Base Clock,1.60 GHz,Base Clock,2.10 GHz,Base Clock,1.00 GHz,Base Clock,2.40 GHz,...,Integrated Graphics,-,Integrated Graphics,-,Integrated Graphics,-,Integrated Graphics,-,Integrated Graphics,-
3,Boost Clock,3.60 GHz,Boost Clock,4.20 GHz,Boost Clock,3.70 GHz,Boost Clock,3.60 GHz,Boost Clock,4.10 GHz,...,Model,GIGABYTE B365 M AORUS ELITE,Model,ID COOLING AURAFLOW X 240 RGB,Model,ASROCK B450 STEEL LEGEND,Model,DEEPCOOL CASTLE 280 ARGB,Model,GIGABYTE B450 AORUS ELITE
4,Cache,4 MB,Cache,6 MB,Cache,4 MB,Cache,6 MB,Cache,8 MB,...,Chipset,B365,Model,MSI Z390-A PRO,Chipset,B450,Model,GIGABYTE Z390 AORUS ELITE,Chipset,B450


In [14]:
triples = []
relations = []

for k,v in df.iteritems():
    idx = 0
    for idx in range(len(v)-1):
        if v[idx+1] is not None:
            

0       CPU Brand
1       CPU Model
2      Base Clock
3     Boost Clock
4           Cache
         ...     
57            NaN
58            NaN
59            NaN
60            NaN
61            NaN
Name: P_0Q, Length: 62, dtype: object
0                   AMD
1     AMD Ryzen 5 3500U
2              2.00 GHz
3              3.60 GHz
4                  4 MB
            ...        
57                  NaN
58                  NaN
59                  NaN
60                  NaN
61                  NaN
Name: P_0A, Length: 62, dtype: object
0       CPU Brand
1       CPU Model
2      Base Clock
3     Boost Clock
4           Cache
         ...     
57            NaN
58            NaN
59            NaN
60            NaN
61            NaN
Name: P_1Q, Length: 62, dtype: object
0                    Intel
1     Intel Core i5-10210U
2                 1.60 GHz
3                 4.20 GHz
4                     6 MB
              ...         
57                     NaN
58                     NaN
59        